In [ ]:
# Parameters (do not containerize this cell)
param_data_filename <- "Template_MBO_Example_raw.xlsx"
param_metadata_sheet <- "METADATA"
param_data_sheet <- "BIRDS"
param_user_email = "[the e-mail address you used to log into NaaVRE]"
param_use_dummy_data <- 1
param_years <- 7
param_latitude_north <- 90.0000
param_latitude_south <- 25.0000
param_longitude_east <- 70.0000
param_longitude_west <- -45.0000
param_upper_limit_max_depth <- 0
param_lower_limit_max_depth <- 50
param_upper_limit_min_depth <- 0
param_lower_limit_min_depth <- 50
param_first_month <- 1
param_last_month <- 3
conf_temporary_data_directory <- "/tmp/data"
conf_virtual_lab_biotisan_euromarec <- "vl-biotisan-euromarec"
conf_minio_endpoint <- "scruffy.lab.uvalight.net:9000"
conf_minio_region <- "nl-uvalight"
conf_minio_public_bucket <- "naa-vre-public"
conf_minio_user_bucket <- "naa-vre-user-data"

In [ ]:
# Secrets (do not containerize this cell)
library("SecretsProvider")

secretsProvider <- SecretsProvider()
secret_minio_access_key = ""
secret_minio_access_key = secretsProvider$get_secret("secret_minio_access_key")
secret_minio_secret_key = ""
secret_minio_secret_key = secretsProvider$get_secret("secret_minio_secret_key")

In [ ]:
# MinIO data retriever
library("readxl")
library("aws.s3")

Sys.setenv("AWS_S3_ENDPOINT" = conf_minio_endpoint,
           "AWS_DEFAULT_REGION" = conf_minio_region,
           "AWS_ACCESS_KEY_ID" = secret_minio_access_key,
           "AWS_SECRET_ACCESS_KEY" = secret_minio_secret_key)

# Download file from bucket S3
if (param_use_dummy_data) {
        file_path <- paste(conf_virtual_lab_biotisan_euromarec, param_data_filename, sep="/")
        print(sprintf("Using dummy data for testing purposes. Set param_use_dummy_data to 0 to use your own data. Downloading data from %s / %s", conf_minio_public_bucket, file_path))
        aws.s3::save_object(bucket=conf_minio_public_bucket, object=file_path, file=param_data_filename)
    } else {
        file_path <- paste(param_user_email, param_data_filename, sep="/")
        print(sprintf("Downloading data from %s / %s", conf_minio_user_bucket, file_path))
        aws.s3::save_object(bucket=conf_minio_user_bucket, object=file_path, file=param_data_filename)
}

# Load data & metadata
metadata <- read_excel(param_data_filename, sheet = param_metadata_sheet) #Load metadata sheet
data <- read_excel(param_data_filename, sheet = param_data_sheet) #Load data sheet

# Ensure the temporary data storage directory exists
dir.create(conf_temporary_data_directory, showWarnings = FALSE)

# Write (meta)data to files
metadata_as_csv_filename <- "metadata.csv"
data_as_csv_filename <- "data.csv"
metadata_from_excel_path <- paste(conf_temporary_data_directory, metadata_as_csv_filename, sep="/")
data_from_excel_path <- paste(conf_temporary_data_directory, data_as_csv_filename, sep="/")
print(sprintf("Storing metadata in: %s, and data in %s", metadata_from_excel_path, data_from_excel_path))
write.csv(metadata, file = metadata_from_excel_path)
write.csv(data, file =  data_from_excel_path)

In [ ]:
install.packages("tidyr")

In [ ]:
# Data Cleaner
library(dplyr)
library(tidyr)

# Assign dummy variables to prevent false Input/Output detection by the NaaVRE cell analyzer
datecollected = ""
siteid = ""

# Read (meta)data from files
md <- read.csv(paste(conf_temporary_data_directory, metadata_as_csv_filename, sep="/"), sep=",")
data <- read.csv(paste(conf_temporary_data_directory, data_as_csv_filename, sep="/"), sep=",")

###### Years filter ######
###### The "number of sampled years" could be changed by the user (default = 7) ######
# Create a table with sites with more than 7 sampling years
sites <- data %>% 
  group_by(siteid) %>%
  summarise(nyear = n_distinct(substr(datecollected, 1, 4))) %>%
  filter(nyear > param_years)

md <- merge(md,sites, by = "siteid")

###### Coordinates filter ######
###### The "geographical boundaries" could be changed by the user (default = latitude (25:90), longitude (-45:70)). The default values correspond to European continental waters. ######
# Keep sites within the study area [our boundaries are latitude (25:90), longitude (-45:70)]
md <- filter(md, decimallatitude >= param_latitude_south, decimallatitude <= param_latitude_north, 
                 decimallongitude >= param_longitude_west, decimallongitude <= param_longitude_east)

data <- data %>% # Keep data from these sites
  filter(siteid %in% md$siteid)

###### Depth filter ######
###### The "depth" could be changed by the user (default = 0 to 50 meters). It should be given in absolute value. ######
#(in this case is not necessary, but for other taxonomic groups is possible that the sample were taken at different depths. The code keeps the NAs in case that information is not known)
data <- data %>% filter((maximumdepthinmeters >= param_upper_limit_max_depth & maximumdepthinmeters <= param_lower_limit_max_depth) %>% replace_na(TRUE))
data <- data %>% filter((minimumdepthinmeters >= param_upper_limit_min_depth & minimumdepthinmeters <= param_lower_limit_min_depth) %>% replace_na(TRUE))



###### Season filter ######
###### The "season" could be changed by the user (default = 1:3). The default values correspond to winter.
# The period does not need to be three months, can be 1:12 for the whole year. It cannot choose months from different years (for example, November to January) ######
# In this case, most of the sampling campaigns were conducted in winter
# One was conducted in summer and should be removed since the sampling season is not consistent
data$month <- as.numeric(format(as.Date(data$datecollected), "%m")) # Create a column with the sampling month

season <- c(param_first_month:param_last_month)
data <- data %>%
  filter(month %in% season) #Remove those samples in non-consistent seasons (in this case keeps the months 1, 2 and 3, this is January, February and March)

# Note that some time series can have more than one sampling campaign per year and even per season (not in this case)
# For our analysis, we are only keeping one sampling campaign per year

###### Years filter ######
###### Note that this step is repeated ######
# Update the table with sites with more than the number of sampled years
# After removing inconsistent sampling campaigns, some time series may become shorter than 8 years
sites <- data %>% 
  group_by(siteid) %>%
  summarise(nyear = n_distinct(substr(datecollected, 1, 4))) %>%
  filter(nyear > param_years)

data <- data %>% # Keep data from these sites
  filter(siteid %in% md$siteid)
md <- md %>% # Keep metadata from these sites
  filter(siteid %in% md$siteid)

md_final <- md[,c(1:8)]
data_final <- data[,c(1:15)]

# Create filenames
cleaned_metadata_filename <- "metadata_Example.csv"
cleaned_data_filename <- "data_Example.csv"

# Write data to files
write.csv(md_final, file = paste(conf_temporary_data_directory, cleaned_metadata_filename, sep="/"))
write.csv(data_final, file = paste(conf_temporary_data_directory, cleaned_data_filename, sep="/"))


In [ ]:
# Trend analyzer
library(vegan)
library(dplyr)
library(ggplot2)
library(nlme)

# Assign dummy variables to prevent false Input/Output detection by the NaaVRE cell analyzer
datecollected = ""
parameter_value = ""

# Load cleaned data & metadata
md <- read.csv(paste(conf_temporary_data_directory, cleaned_metadata_filename, sep="/"), sep=",")
data <- read.csv(paste(conf_temporary_data_directory, cleaned_data_filename, sep="/"), sep=",")
data$year <- as.numeric(format(as.Date(data$datecollected), "%Y"))
colnames(data)

# Calculate community metrics
data.tax <- data %>%
  group_by(siteid, year, datecollected) %>%
  summarise(richness = n_distinct(taxaname[parameter_value > 0]), # Richness
            parameter_value_tot = sum(parameter_value), # Abundance estimate
            parameter = unique(parameter),
            parameter_standardunit = unique(parameter_standardunit),
            diversity = diversity(parameter_value, index="shannon"), # Diversity
            )

# Temporal analysis. Example with Richness and these 2 time series
results.richness <- data.frame(siteid = character(0), slope = numeric(0), p = numeric(0))

for (i in names(table(data.tax$siteid))) {
  x <- subset(data.tax, siteid == i)
  # We used GLS models taking into account the temporal autocorrelation
  gls_model <- gls(log10(richness+1) ~ year, data = x, correlation = corAR1(form = ~ 1 | year))
  slope <- coef(gls_model)[2]
  p <- summary(gls_model)$tTable[2, 4]

  # Save results
  results.richness <- rbind(results.richness, data.frame(siteid = i, slope = slope, p = p))
}

print(results.richness)

# In this example the second site showed a significant decrease in Richness over time (p<0.05)

final_results <- merge(md,results.richness, by = "siteid")
print(final_results)

# Store results as CSV to temporary file
results_filename <- "biodiversity_trend_analysis_results.csv"
results_file_path <- paste(conf_temporary_data_directory, results_filename, sep="/")
print(sprintf("Storing final_results to %s", results_file_path))
write.csv(final_results, file = results_file_path)

In [ ]:
# Output storer
Sys.setenv("AWS_S3_ENDPOINT" = conf_minio_endpoint,
           "AWS_DEFAULT_REGION" = conf_minio_region,
           "AWS_ACCESS_KEY_ID" = secret_minio_access_key,
           "AWS_SECRET_ACCESS_KEY" = secret_minio_secret_key)

# Upload file to bucket
put_object(bucket=conf_minio_user_bucket, file=paste(conf_temporary_data_directory, results_filename, sep="/"), object=paste(param_user_email, results_filename, sep="/"))